In [1]:
import couchdb
import elasticsearch
from elasticsearch.helpers import scan

In [13]:
# Set ES connection
es = elasticsearch.Elasticsearch('http://10.10.100.48/')
es.ping()

True

In [14]:
# Set CouchDB connection
couch = couchdb.Server('http://10.10.100.48/couchdb')

In [15]:
all_email_array = []
not_all_email = []
error = []

In [16]:
docs = scan(es, query={"query": {"match_all": {}}})

for doc in docs:
    try:
        doc_id = doc['_id']
        doc_index = doc['_index']
        contact = doc['_source']['metadata']['contact']
        contact_array = contact.split(';')
        contact_array = [unicode(person.strip()) for person in contact_array]
        # Check if contact field has only emails
        all_email = True
        for email in contact_array:
            if '@' not in email and email != '':
                all_email = False
                break
        if not all_email:
            not_all_email.append((doc_index, doc_id))
        else:
            all_email_array.append((doc_index, doc_id))
    except Exception:
        error.append((doc_index, doc_id))

In [17]:
import sqlite3

db = sqlite3.connect("users.sqlite")
c = db.cursor()

In [25]:
all_contacts = []
not_found = []
too_many = []
ok = []
ok2 = []
not_found2 = []
too_many2 = []
ok3 = []
add_email = []
for doc in not_all_email:
    update = False
    # Get alias or index name
    aliases = es.indices.get_aliases(index=doc[0])
    if len(aliases[doc[0]]["aliases"]) == 0:
        doc_index = doc[0]
    else:
        doc_index = aliases[doc[0]]["aliases"].keys()[0]
    doc_id = doc[1]
    
    # Get document
    db = couch[doc_index]
    doc_couch = db.get(doc_id)

    contributor = doc_couch['metadata']['contributor']
    contact = doc_couch['metadata']['contact']
    contributor_array = contributor.split(';')
    contact_array = contact.split(';')
    contributor_array = [unicode(person.strip()) for person in contributor_array]
    contact_array = [unicode(person.strip()) for person in contact_array]
    contact_copy = list(contact_array)
    
    for name in contact_copy:
        # Nome não é email e não está listado
        if '@' not in name:
            if name not in contributor_array:
                # Procura pelo nome
                c.execute('''select * from users where name='%s' ''' % name)
                row = c.fetchall()
                if not row:
                    not_found.append(name)
                elif len(row) > 1:
                    too_many.append(name)
                else:
                    ok2.append(name)
            else:
                # Verifica se email também está
                c.execute('''select * from users where name='%s' ''' % name)
                row = c.fetchall()
                
                if not row:
                    not_found2.append(name)
                elif len(row) > 1:
                    too_many2.append(name)
                else:
                    ok3.append(name)
                
                if name == 'Luiz Santos':
                    email = 'luiz@cncflora.net'
                elif name == 'Diogo Silva':
                    email = 'diogo@cncflora.jbrj.gov.br'
                else:
                    email = row[0][1]
                cont_antes = list(contact_array)
                # Remove nome de contact_array
                ix = contact_array.index(name)
                del contact_array[ix]
                if email not in contact_array:
                    # Adiciona email
                    contact_array.insert(ix, email)
                    add_email.append((doc_index, doc_id, email, name, contributor_array, contact_array, cont_antes))
                else:
                    ok.append((doc_index, doc_id, email, name, contributor_array, contact_array, cont_antes))
                update = True
            
    all_contacts.append(contact_array)
    doc_couch['metadata']['contact'] = " ; ".join(contact_array)
    #if update:
        #db.update([doc_couch])    

In [26]:
print len(not_found)
print len(too_many)
print len(ok2)
print len(ok)
print len(add_email)

0
0
0
0
0


In [137]:
set(not_found)

{u'Luiz Santos'}

In [23]:
not_all_email

[(u'novas_especies_2015_08_26_19:29', u'1b951bed-4ccc-4d98-85c0-002d8c3d33b2')]